In [3]:
from tensorflow.keras.applications.resnet50 import preprocess_input   #gereki olan kütüphaneleri ve paketleri tanımlıyoruz
from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.preprocessing.image import image_to_array
from tensorflow.keras.utils import img_to_array
from keras.applications import imagenet_utils
import numpy as np
import cv2
from detector.sliding_window import sliding_window
from detector.image_pyramid import image_pyramid
from detector.non_max_suppression import non_maxi_suppression

#ilkelendirme parametreleri 
WIDTH = 600  #genişlik 
HEIGHT = 600  #yükseklik
PYR_SCLAE = 1.5 #scale değeri
WIN_STEP = 16 #kaç piksel atlayacağımızın sayısı
ROI_SIZE = (200,150) #dikdörtgen boyutu
INPUT_SIZE = (224,224) #Resnet mimarisine sokacağımız resmin girdi boyutları 

print('Resnet yukleniyor')

model =ResNet50(weights = 'imagenet',include_top = True) #ImageNet = veri kümesi ile eğitilmiş ağırlıkları kullanıyoruz 

orig = cv2.imread('src/husky2.jpg') #resmimizi diskten alıyourz
orig = cv2.resize(orig,(WIDTH,HEIGHT))  #yukarda belirttiğimiz değerler ile yeniden boyutlandırıyoruz

(H,W) = orig.shape[:2] #ve bu değerleri H ve W değişkenine atıyoruz

#image pyramid 
pyramid = image_pyramid(orig,PYR_SCLAE,ROI_SIZE)

rois = []  #roi leri eklemek için boş bir liste oluşturuyoruz
locs = []

for image in pyramid:  # image_pyramid fonksiyonuyla oluşmuş resimlerde geziniyoruz
    scale = W/float(image.shape[1])  #scale değeri belirliyoruz

    for (x,y,roiOrig) in sliding_window(image,WIN_STEP,ROI_SIZE):  #sliding_window fonksiyonuyla oluşturulmuş resimler içinde geziniyoruz
        x = int(x*scale)  # x ve y değerlerini yukarıda belirlediğimiz scale değeriyle çarpıyoruz
        y = int(y*scale)
        w = int(ROI_SIZE[0]*scale)  #w değerini 200
        h = int(ROI_SIZE[1]*scale)  #h değerini ise 150 ile çarpıyoruz

        roi = cv2.resize(roiOrig,INPUT_SIZE)  #gezindiğimiz resimleri yeniden boyutlandırıyoruz 224x224
        roi = img_to_array(roi)  #array formuna çeviriyoruz
        roi = preprocess_input(roi)  #resimleri resnet50 modeline uygun hale getiriyoruz 

        rois.append(roi)  #resimler rois listesine eklenir 
        locs.append((x,y,x+w,y+h))  # sol üst köşe ve sağ alt köşe koordinatları ise locs listesine eklenir

rois = np.array(rois,dtype = 'float') #rois içindeki resimleri tipi float olacak şekilde numpy array formatına getiriyoruz

preds = model.predict(rois) #tahminleri dönderiyoruz
preds = imagenet_utils.decode_predictions(preds,top = 1)

#print(preds)

labels = {}  # olasılık değerleri 0.9 ' dan yüksek olan sınıfları bu sözlük içine ekliyoruz
for (i,p) in enumerate(preds):

    (_,label,prob) = p[0]
    if prob >= 0.9:
        box = locs[i]
        L = labels.get(label,[]) 
        L.append((box,prob))
        labels[label] = L

#print(labels.keys()) => # dict_keys(['eskimo_dog'])
for label in labels.keys():  #tekrarlayan sınırlayıcı kutuları önlemek için 
    clone = orig.copy()
    for (box,prob) in labels[label]:
        (startX,startY,endX,endY) = box
        cv2.rectangle(clone,(startX,startY),(endX,endY),(0,255,0),2)
    #cv2.imshow('ilk',clone)

    clone = orig.copy()  #orijinal resmimizin bozulmaması için  clone adlı bir kopya oluşturuyoruz

    boxes = np.array([p[0] for p in labels[label]])
    proba = np.array([p[1] for p in labels[label]])

    boxes = non_maxi_suppression(boxes,proba)  #tekrarlayan sınırlayıcı kutuları önlüyoruz
 
    for (startX,startY,endX,endY) in boxes: #önlenmiş sınırlayıcı kutular ve etiketleri clone görüntüsü üzerine çiziyoruz ve sınıflandırılmış nesnelerin etiketlerini ilgili sınırlayıcı kutuların yanına yazıyoruz
        cv2.rectangle(clone,(startX,startY),(endX,endY),(0,255,0),2)
        y = startY - 10 if startY-10 > 10 else startY+10
        cv2.putText(clone,label,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.45,(0,255,0),2)
    cv2.imshow('Maxima',clone)  #clone resmimizi imshowluyoruz

    if cv2.waitKey(1) == ord('q'): # eğer ki 'q' ya basılırsa döngüyü kırıyoruz
        break

cv2.waitKey(0)  
cv2.destroyAllWindows()




Resnet yukleniyor
31/31 [==============================] - 101s 3s/step
